In [1]:
execfile('ipython/00-bdd-shell-init.py')

Checkpoint4
Checkpoint5
<class 'pyspark.conf.SparkConf'>
[(u'spark.app.name', u'BDD-Shell'), (u'spark.yarn.jar', u'local:///home/bdd/Oracle/Middleware/BDD-1.3.2.35.1091/common/bdd-shell/lib/spark-assembly-1.6.0-hadoop2.6.0.jar'), (u'spark.executor.memory', u'1g'), (u'spark.executor.instances', u'2'), (u'spark.master', u'yarn-client'), (u'spark.executor.cores', u'1'), (u'spark.submit.deployMode', u'client'), (u'spark.driver.memory', u'1g'), (u'spark.driver.cores', u'1')]
Checkpoint6
Checkpoint7
Checkpoint10
Checkpoint11


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
get_ipython().magic('matplotlib inline')
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import _arma_predict_out_of_sample
from statsmodels.tsa.stattools import acf, pacf, adfuller
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import acf, pacf

In [3]:
dss = bc.datasets()
dss.count

36

In [4]:
for ds in dss:
    print ds


20161114SFT	default_edp_6ad66d8f-bbdd-4793-b337-5e2cf041e60a	default_edp_6ad66d8f-bbdd-4793-b337-5e2cf041e60a	Hive	default.20161114sft

Test BDD Small	default_edp_19fd2466-c249-4689-9cf0-4ce140de78a9	default_edp_19fd2466-c249-4689-9cf0-4ce140de78a9	Hive	default.test_bdd_small_3

Test BDD Small 5	default_edp_9fff1c6d-3a0e-4b9d-8870-265a1f750282	default_edp_9fff1c6d-3a0e-4b9d-8870-265a1f750282	Hive	default.test_bdd_small_5

SA_ALLOCATION_LINE	default_edp_8e54b84d-e55e-4e1c-a1b1-aa168e6ea5cd	default_edp_8e54b84d-e55e-4e1c-a1b1-aa168e6ea5cd	Hive	default.sa_allocation_line

GSP_OOCL_CALENDAR	default_edp_22982791-8157-4ae0-bc4c-eb7925818798	default_edp_22982791-8157-4ae0-bc4c-eb7925818798	Hive	default.gsp_oocl_calendar

SWAT_OPZONE_LAT_LON	default_edp_e4b09f41-ac0e-4602-86c7-0f04c7653d40	default_edp_e4b09f41-ac0e-4602-86c7-0f04c7653d40	Hive	default.swat_opzone_lat_lon

20161114SFT	default_edp_a588a16e-3b8f-41dd-aa69-63112bae2293	default_edp_a588a16e-3b8f-41dd-aa69-63112bae2293	Hive	default.

In [5]:
# Load Requested Opzone List

ds_opzone=dss.dataset('default_edp_0997225b-b9e7-4aea-a88f-0ce77b21ddb7')
#ds.properties()

# Data source to Spark DataFrame to Pandas DataFrame
df_opzone = ds_opzone.to_spark()
pdf_opzone = df_opzone.toPandas()

# Convert to array
#pdf_opzone = pdf_opzone[pdf_opzone.opzone_cde != 'ZHJ']
pdf_opzone = pdf_opzone[pdf_opzone.opzone_cde == 'ZHK']
opzones_list_tmp = pdf_opzone[['opzone_cde']]
opzones_list = opzones_list_tmp['opzone_cde'] 


In [6]:
# Load MT_RLS and MT_RTN Actual List

ds_mt=dss.dataset('default_edp_dec74e08-a3c1-47a5-bafb-8fa8dfdadd6c')
#ds.properties()

# Data source to Spark DataFrame to Pandas DataFrame
df_mt = ds_mt.to_spark()
pdf_mt = df_mt.toPandas()

In [7]:
import datetime
def date_convert_fn(x):
    return datetime.datetime.fromtimestamp(x/1000.0).strftime('%Y-%m-%d')

pdf_mt['date'] = pdf_mt.apply(lambda row: date_convert_fn(row['date']), axis=1)

In [8]:
pdf_mt.set_index(keys=['date'], drop=True,inplace=True)
pdf_mt.index=pd.to_datetime(pdf_mt.index)
pdf_mt.head(5)

,PRIMARY_KEY,eqp_size_type,mt_rls,mt_rtn,opzone_cde
date,,,,,
2011-12-31,0-0-0,20GP,26,12,ZAA
2012-01-07,0-0-1,20GP,19,3,ZAA
2012-01-14,0-0-2,20GP,13,7,ZAA
2012-01-21,0-0-3,20GP,10,13,ZAA
2012-01-28,0-0-4,20GP,43,7,ZAA


In [9]:
# ARIMA Time Series
for opzone in opzones_list:
    sizetype = '40GP'
    df = pdf_mt[ (pdf_mt.opzone_cde==opzone) & (pdf_mt.eqp_size_type == sizetype) ]
    df_RLS = df[['mt_rls']]
    df_RTN = df[['mt_rtn']]
    df_RLS.sort_index(inplace=True, ascending=True)
    df_RTN.sort_index( inplace=True, ascending=True)
    df_RLS_TS = df_RLS['mt_rls'] 
    df_RTN_TS = df_RTN['mt_rtn']
    result = pd.DataFrame(columns=('1wk', '2wk', '3wk','4wk', '5wk'))
    
    for n in range(0,53):
        y = df_RTN_TS[0 :-53 + n]

        try:
            res = ARIMA(y, order=(2, 1, 2)).fit(trend="nc")
        except:
            try:
                res = ARIMA(y, order=(2, 1, 1)).fit(trend="nc")
            except:
                try:
                    res = ARIMA(y, order=(1, 1, 2)).fit(trend="nc")
                except:
                    try:
                        res = ARIMA(y, order=(1, 1, 1)).fit(trend="nc")
                    except:
                        print "invertiable error occured."
    
        predication = res.forecast(5)[0]
        result.loc[n] = [predication[0],predication[1],predication[2],predication[3],predication[4]] 
        
    result['OPZONE_CODE'] = opzone
    result['SIZE_TYPE'] = sizetype
    df_temp = pd.DataFrame(df_RTN_TS[-53:])
    result.index = df_temp.index
    df_temp1 = pd.concat([result, df_temp], axis=1)
    df_temp1['2wk'] = df_temp1['2wk'].shift(1)
    df_temp1['3wk'] = df_temp1['2wk'].shift(2)
    df_temp1['4wk'] = df_temp1['2wk'].shift(3)
    df_temp1['5wk'] = df_temp1['2wk'].shift(4)
    df_temp1.head
    #df_temp1.to_csv('./dataout/export_arima_40GP_MTRTN.csv', mode='a', header=False)

In [10]:
df_temp1.head(5)

,1wk,2wk,3wk,4wk,5wk,OPZONE_CODE,SIZE_TYPE,mt_rtn
date,,,,,,,,
2015-09-26,337.748480,NaN,NaN,NaN,NaN,ZHK,40GP,253
2015-10-03,325.323770,339.276860,NaN,NaN,NaN,ZHK,40GP,324
2015-10-10,329.124422,329.339095,NaN,NaN,NaN,ZHK,40GP,326
2015-10-17,323.829777,324.350682,339.276860,NaN,NaN,ZHK,40GP,356
2015-10-24,326.362056,320.983390,329.339095,339.27686,NaN,ZHK,40GP,268


In [11]:
# Convert the output Pandas DataFrame to Spark DataFrame
sdf = sqlContext.createDataFrame(df_temp1)

In [12]:
# Persist the Spark DataFrame into new Hive table
sdf.write.saveAsTable('default.ARIMA_ZHK_40GP_MTRTN_3')

In [13]:
sdf.printSchema()

root
 |-- 1wk: double (nullable = true)
 |-- 2wk: double (nullable = true)
 |-- 3wk: double (nullable = true)
 |-- 4wk: double (nullable = true)
 |-- 5wk: double (nullable = true)
 |-- OPZONE_CODE: string (nullable = true)
 |-- SIZE_TYPE: string (nullable = true)
 |-- mt_rtn: long (nullable = true)



In [14]:
#import from HIVE to BDD
from subprocess import call
call(["/home/bdd/Oracle/Middleware/BDD-1.3.2.35.1091/dataprocessing/edp_cli/data_processing_CLI", "-t ARIMA_ZHK_40GP_MTRTN_3"])

0